# Tutorial 3: Parameter Estimation (Inverse Problems)

Learn how to fit model parameters to observed data using MechanicsDSL's inverse problem API.

In [ ]:
from mechanics_dsl import PhysicsCompiler
from mechanics_dsl.inverse import ParameterEstimator, SensitivityAnalyzer
import matplotlib.pyplot as plt
import numpy as np

## Generate Synthetic Data

First, we simulate a system with known parameters, then add noise to create "observations".

In [ ]:
dsl_code = r"""
\system{spring_mass}
\defvar{x}{Position}{m}
\defvar{m}{Mass}{kg}
\defvar{k}{Spring Constant}{N/m}

\parameter{m}{2.5}{kg}    % True value we want to recover
\parameter{k}{15.0}{N/m}  % True value we want to recover

\lagrangian{
    \frac{1}{2} * m * \dot{x}^2 - \frac{1}{2} * k * x^2
}

\initial{x=1.0, x_dot=0.0}
"""

# Generate "true" solution
compiler = PhysicsCompiler()
compiler.compile_dsl(dsl_code)
true_solution = compiler.simulate(t_span=(0, 5), num_points=50)

t_obs = true_solution['t']
observations = true_solution['y'][0] + np.random.normal(0, 0.02, len(t_obs))

plt.figure(figsize=(10, 4))
plt.plot(t_obs, true_solution['y'][0], 'b-', label='True')
plt.scatter(t_obs, observations, c='red', s=20, alpha=0.6, label='Observations')
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.legend()
plt.title('Synthetic Observations with Noise')
plt.show()

## Fit Parameters

Now pretend we don't know the true parameters. Use `ParameterEstimator` to find them.

In [ ]:
# Reset parameters to wrong initial guess
compiler.simulator.set_parameters({'m': 1.0, 'k': 10.0})

# Create estimator and fit
estimator = ParameterEstimator(compiler)

result = estimator.fit(
    observations=observations.reshape(-1, 1),
    t_obs=t_obs,
    params_to_fit=['m', 'k'],
    bounds={'m': (0.5, 10.0), 'k': (5.0, 50.0)},
    method='L-BFGS-B'
)

print(f"Fitting success: {result.success}")
print(f"True m=2.5, Fitted m={result.parameters['m']:.3f}")
print(f"True k=15.0, Fitted k={result.parameters['k']:.3f}")
print(f"Residual: {result.residual:.6f}")

In [ ]:
# Compare fitted solution to observations
compiler.simulator.set_parameters(result.parameters)
fitted_solution = compiler.simulate(t_span=(0, 5), num_points=200)

plt.figure(figsize=(10, 4))
plt.scatter(t_obs, observations, c='red', s=20, alpha=0.6, label='Observations')
plt.plot(fitted_solution['t'], fitted_solution['y'][0], 'g-', lw=2, label='Fitted model')
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.legend()
plt.title('Fitted Model vs Observations')
plt.show()

## Sensitivity Analysis

Understand which parameters have the biggest impact on model output.

In [ ]:
analyzer = SensitivityAnalyzer(compiler)

indices = analyzer.sobol_analysis(
    param_ranges={'m': (1.0, 5.0), 'k': (5.0, 30.0)},
    n_samples=128
)

print("First-order Sobol indices (main effects):")
for param, index in indices.first_order.items():
    print(f"  {param}: {index:.3f}")

print("\nTotal-order indices (including interactions):")
for param, index in indices.total_order.items():
    print(f"  {param}: {index:.3f}")

In [ ]:
# Visualize sensitivity
params = list(indices.first_order.keys())
first = [indices.first_order[p] for p in params]
total = [indices.total_order[p] for p in params]

x = np.arange(len(params))
width = 0.35

fig, ax = plt.subplots(figsize=(8, 5))
ax.bar(x - width/2, first, width, label='First-order')
ax.bar(x + width/2, total, width, label='Total-order')
ax.set_ylabel('Sensitivity Index')
ax.set_xticks(x)
ax.set_xticklabels(params)
ax.legend()
ax.set_title('Sobol Sensitivity Indices')
plt.show()